In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.ticker
import matplotlib.ticker as ticker
from matplotlib.ticker import AutoMinorLocator
from pylab import *
import os.path
import tables

import matplotlib.cm as cm
from matplotlib.colors import LogNorm
import matplotlib as mpl
from scipy.signal import butter, lfilter
from astropy.io import fits
from scipy import ndimage as ndi
from scipy import stats
from collections import OrderedDict

import sys; sys.path.insert(0, '/home/hgan/ch-env/code/minisom/')
import minisom # store_load is a file on my library folder
from minisom import MiniSom    

import pandas as pd
#import sys; sys.path.insert(0, '/home/hgan/ch-env/code/seaborn')
import seaborn as sn

import time
import scipy
import csv
import os
import random

import copy 
from matplotlib.colors import ListedColormap
from pathlib import Path
import sys; sys.path.insert(0, '/home/hgan/ch-env/code/chimedb/')
import chimedb
from tqdm.notebook import tqdm

from mpi4py import MPI

import pickle


In [2]:
####### functions 

def trim_data(tmp):
    rep = tmp.replace(' -inf', 0)#np.nan)
    rep = rep.replace(' inf', 0) #np.nan)
    rep = rep.replace(' -nan ', 0)#np.nan)
    rep = rep.replace(' nan ', 0)#np.nan)
    rep = rep.replace(np.nan, 0)
    rep = rep.astype(float)
    return rep 
   
def get_stats(array):
    array = np.ma.masked_invalid(array)
    return np.sum(array), np.mean(array), np.std(array)
   
revision = "rev_07"      
training_no = "train1"
data_dir = '/project/rpp-chime/hgan/ML-analysis/trees/'+revision+'/'
base_directory='/project/rpp-chime/hgan/ML-analysis_rev07/'
save_directory = base_directory + training_no+'/'
tree_directory = '/project/rpp-chime/hgan/ML-analysis/trees/'+revision+'/'
som_directory = base_directory + training_no+'/SOMs/'

##### training set

daily_processing_path = Path(f"/project/rpp-chime/hgan/ML-analysis/delaysp/{revision}/")
obs_set = list(daily_processing_path.iterdir())
id_set = []

for obs in obs_set:
    s = str(obs)
#    print(s[-8:-4])
    try: 
        id_set.append(int(s[-9:-5]))
    except:
        continue
        
id_set = sorted(id_set)

###### load training data

print('.........Start loading data sets........')

sample_percentage = .05 
file_names = []

ext = ('.csv')
  
###### training              

s_features = [4,5,6,8,9,13,14] #10,11]#,13,14] #[12,13,14,16,17,18,19,20,21]

# Initialization
m_neurons = 10 
n_neurons = 10 

sigma = 1.
learning_rate = .2 
neighborhood_function = 'gaussian'
random_seed = 0
topology = 'rectangular'
activation_distance = 'euclidean'

#### Open a SOM
with open(save_directory+'som-'+training_no+'.p', 'rb') as infile:
    som = pickle.load(infile)
    
print('.........training finished, start calculating BMUs........')

###### calculate BMUs

# comm = MPI.COMM_WORLD
# rank = comm.Get_rank()
# size = comm.Get_size()

# a = 0
# b = len(id_set) 

# num_per_rank = b // size # the floor division // rounds the result down to the nearest whole number.

# lower_bound = a + rank * num_per_rank
# upper_bound = a + (rank + 1) * num_per_rank




.........Start loading data sets........
.........training finished, start calculating BMUs........


In [3]:
print(id_set)

[1868, 1869, 1870, 1871, 1872, 1874, 1878, 1879, 1880, 1885, 1886, 1889, 1890, 1892, 1893, 1894, 1899, 1902, 1903, 1904, 1906, 1907, 1908, 1909, 1910, 1912, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922, 1924, 1926, 1927, 1928, 1929, 1931, 1932, 1933, 1934, 1935, 1936, 1939, 1955, 1956, 1958, 1959, 1960, 1965, 1966, 1967, 1969, 1970, 1971, 1972, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1988, 1989, 1990, 2012, 2013, 2029, 2030, 2031, 2032, 2033, 2039, 2040, 2041, 2043, 2044, 2045, 2046, 2047, 2059, 2060, 2072, 2073, 2074, 2075, 2077, 2079, 2080, 2081, 2084, 2085, 2086, 2087, 2093, 2094, 2095, 2096, 2097, 2098, 2099, 2105, 2106, 2107, 2108, 2109, 2110, 2111, 2112, 2115, 2116, 2118, 2119, 2120, 2121, 2122, 2123, 2126, 2127, 2132, 2133, 2134, 2135, 2142, 2143, 2163, 2164, 2166, 2167, 2174, 2175, 2179, 2180, 2181, 2182, 2186, 2193, 2207, 2228, 2229, 2242, 2243, 2249, 2256, 2257, 2277, 2278, 2291, 2292, 2299, 2305, 2306, 2320, 2325, 2347, 2348, 235

In [4]:
print(lower_bound, upper_bound)

NameError: name 'lower_bound' is not defined

In [3]:
for num_i in range(0, 50):
    iid = id_set[num_i]
    name = 'csd'+str(iid)+'_tree.csv'
    print("Calculating...." + name)
    single_data = pd.read_csv(tree_directory + name, sep=',', header=0, low_memory=False) 
    single_data = trim_data(single_data)
    features = np.array(list(single_data.keys()))

    n_columns = np.shape(single_data)[-1]
    flat_data = []

    for i in range(0,len(features)):
        flat_data.append(single_data.iloc[:,i].to_numpy())

    X = np.array(flat_data)[s_features]
    X = np.array(X).T
    scaled_X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)  

    W = som.get_weights()
    w_x, w_y = zip(*[som.winner(d) for d in scaled_X])
    w_x = np.array(w_x)
    w_y = np.array(w_y)  

    map_size=str(n_neurons)+'x'+str(m_neurons)

    df2 =  pd.DataFrame({'index': flat_data[0],
           'winning_Nx': w_x,
           'winning_Ny': w_y,
                        })
    os.makedirs(som_directory, exist_ok=True)  
    df2 = df2.loc[:, ~df2.columns.str.contains('^Unnamed')]
    df2 = df2.astype(int)
    df2.to_csv(som_directory+str(iid)+'_'+map_size+'SOMs.csv')  

print('.......DONE!!!........')

Calculating....csd1868_tree.csv
Calculating....csd1869_tree.csv
Calculating....csd1870_tree.csv
Calculating....csd1871_tree.csv
Calculating....csd1872_tree.csv
Calculating....csd1874_tree.csv
Calculating....csd1878_tree.csv
Calculating....csd1879_tree.csv
Calculating....csd1880_tree.csv
Calculating....csd1885_tree.csv
Calculating....csd1886_tree.csv
Calculating....csd1889_tree.csv
Calculating....csd1890_tree.csv
Calculating....csd1892_tree.csv
Calculating....csd1893_tree.csv
Calculating....csd1894_tree.csv
Calculating....csd1899_tree.csv
Calculating....csd1902_tree.csv
Calculating....csd1903_tree.csv
Calculating....csd1904_tree.csv
Calculating....csd1906_tree.csv
Calculating....csd1907_tree.csv
Calculating....csd1908_tree.csv
Calculating....csd1909_tree.csv
Calculating....csd1910_tree.csv
Calculating....csd1912_tree.csv
Calculating....csd1913_tree.csv
Calculating....csd1914_tree.csv
Calculating....csd1915_tree.csv
Calculating....csd1916_tree.csv
Calculating....csd1917_tree.csv
Calculat

In [9]:
print(som_directory+str(iid)+'_'+map_size+'SOMs.csv')

/project/rpp-chime/hgan/ML-analysis_rev07/train1/SOMs/1868_10x10SOMs.csv


In [ ]:

par1 = 'gval'
par2 = 'flux'
sum_table = pd.DataFrame({
                'csd': [],
                'winning_Nx': [],
                'winning_Ny': [],
                'sum_'+par1:[], 
                'sum_'+par2:[]},    index=[])


ext = ('.csv')
a = 0

lower= 0
upper= 335
som_size = 10

for num_i in tqdm(range(lower, upper)):
    obs = id_set[num_i]
    attributes = pd.read_csv(tree_directory 
                             + '/csd'+str(obs)+'_tree.csv', sep=',', header=0, low_memory=False) 
    winning_neurons = pd.read_csv(som_directory+str(obs)+'_'+map_size+'SOMs.csv',
                                  sep=',', header=0, low_memory=False) 

    flux = pd.DataFrame({
                    'winning_Nx': [],
                    'winning_Ny': [],
                    'sum_par1':[],
                    'sum_par2':[]}, index=[])
    img_par1 = np.zeros(som_size*som_size).reshape((som_size, som_size))
    img_par2 = np.zeros(som_size*som_size).reshape((som_size, som_size))


    for nx in range(0,som_size):
        for ny in range(0,som_size):
            mask1 = winning_neurons['winning_Nx'] == nx
            mask2 = winning_neurons['winning_Ny'] == ny

            selected1 = attributes[mask1&mask2][' '+par1]
            selected2 = attributes[mask1&mask2][' '+par2]

            selected1 = np.nan_to_num(selected1)
            selected2 = np.nan_to_num(selected2)

            selected1 = selected1.astype(float)
            selected2 = selected2.astype(float)

            sum_par1= selected1.sum(axis = 0)#, skipna = True)
            sum_par2= selected2.sum(axis = 0)#, skipna = True)
            
            img_par1[9-nx,ny] = sum_par1
            img_par2[9-nx,ny] = sum_par2


            tmp2 =  pd.DataFrame({
                'csd': obs,
                'winning_Nx': nx,
                'winning_Ny': ny,
                'sum_'+par1: sum_par1,  
                'sum_'+par2: sum_par2}, index=[a])
            a = a + 1
            sum_table = pd.concat([sum_table,tmp2], ignore_index=True)
            
    fig, ax = plt.subplots(1, 2, figsize=(9.5, 4.))

    im1 = ax[0].imshow((img_par1), extent=[0,som_size,som_size,0])#,  interpolation='none', )
                  #  aspect='equal',norm=LogNorm(),
                  # vmax=vmax, vmin=vmin,)
    ax[0].set_xlabel('Neuron_x')
    ax[0].set_ylabel('Neuron_y')
    ax[0].set_title(par1)
    #cbar = fig.colorbar(im, orientation='vertical',fraction=0.046, pad=0.04)
    cbar1 = plt.colorbar(im1, ax=ax[0])
    cbar1.set_label(par1)
    cbar1.formatter.set_powerlimits((0, 0))

    im2 = ax[1].imshow((img_par2), extent=[0,som_size,0,som_size])#,  interpolation='none', )
    cbar2 = plt.colorbar(im2, ax=ax[1])
    cbar2.set_label(par2)
    cbar2.formatter.set_powerlimits((0, 0))
    ax[1].set_xlabel('Neuron_x')
    ax[1].set_ylabel('Neuron_y')
    ax[1].set_title(par2)
    os.makedirs(save_directory + 'sum_tables/', exist_ok=True)
    fig.suptitle('csd'+str(obs))
    fig.subplots_adjust(left=0.08, bottom=0.13, right=0.95, top=0.9, wspace=0.26, hspace=0.1)
    fig.savefig(save_directory + 'sum_tables/'+ str(obs)+'_summary.pdf')
    fig.clf()

os.makedirs(save_directory + 'sum_tables/', exist_ok=True)        
sum_table.to_csv(save_directory + 'sum_tables/sum_table.csv')

  0%|          | 0/335 [00:00<?, ?it/s]

/tmp/ipykernel_257121/610989908.py:64: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(1, 2, figsize=(9.5, 4.))


In [3]:
def get_stats(array):
    array = np.ma.masked_invalid(array)
    return np.sum(array), np.mean(array), np.std(array)

In [5]:
par1 = 'gval'
par2 = 'flux'

som_size=10
flux = pd.DataFrame({
                'winning_Nx': [],
                'winning_Ny': [],
                'sum_'+par1:[], 
                'avg_'+par1:[],
                'std_'+par1:[], 
                'sum_'+par2:[],
                'avg_'+par2:[], 
                'std_'+par2:[],
}, index=[])

img1_avg = np.zeros(som_size*som_size).reshape((som_size, som_size))
img1_std = np.zeros(som_size*som_size).reshape((som_size, som_size))

img2_avg = np.zeros(som_size*som_size).reshape((som_size, som_size))
img2_std = np.zeros(som_size*som_size).reshape((som_size, som_size))

for nx in tqdm(range(0,som_size)):
    for ny in range(0,som_size):
        directory = save_directory + 'sum_tables/'

        f = os.path.join(directory, 'sum_table.csv')

        if os.path.isfile(f):
            sum_table = pd.read_csv(f)
    
            xmask = sum_table['winning_Nx'] == nx
            ymask = sum_table['winning_Ny'] == ny
            selected_par1 = sum_table[xmask&ymask]['sum_'+par1]
            selected_par2 = sum_table[xmask&ymask]['sum_'+par2]
            sum1, avg1, std1 = get_stats(selected_par1)
            sum2, avg2, std2 = get_stats(selected_par2)
             
            tmp = pd.DataFrame({
                'winning_Nx': nx,
                'winning_Ny': ny,
                'sum_'+par1: sum1, 
                'avg_'+par1: avg1,
                'std_'+par1: std1, 
                'sum_'+par2: sum2,
                'avg_'+par2: avg2, 
                'std_'+par2: std2,
            },
            index=[nx*som_size + ny])
        else:
            print("sum table not found!")
            break
            
        img1_avg[9-nx,ny] = avg1
        img1_std[9-nx,ny] = std1
        img2_avg[9-nx,ny] = avg2
        img2_std[9-nx,ny] = std2

        flux = pd.concat([flux, tmp], ignore_index=True) 

flux.to_csv(directory+'PS_stats.csv') 

  0%|          | 0/10 [00:00<?, ?it/s]

In [6]:
def calculate_normalised_excess(obsv, sum_table, PS_stats, par, nx, ny):
    xmask = PS_stats['winning_Nx'] == nx
    ymask = PS_stats['winning_Ny'] == ny
    avg = PS_stats[xmask&ymask]['avg_'+par].values
    std = PS_stats[xmask&ymask]['std_'+par].values 
    #print(avg, std)
    
    idmask = sum_table['csd'] == obsv
    xxmask = sum_table['winning_Nx'] == nx
    yymask = sum_table['winning_Ny'] == ny
    
    sum_par = sum_table[idmask&xxmask&yymask]['sum_'+par].values
    np.seterr(invalid='ignore')
    try:
        excess = np.divide((sum_par - avg), std)
    except: 
        excess =  0
    return excess

In [8]:
### normalised excess reponse table 

par1 = 'gval'
par2 = 'flux'

excess = pd.DataFrame({
                'csd': [],
                'winning_Nx': [],
                'winning_Ny': [],
                'excess_'+par1:[], 
                'excess_'+par2:[],

}, index=[])



PS_stats_dir = save_directory + 'sum_tables/PS_stats.csv'
sum_table_dir = save_directory + 'sum_tables/sum_table.csv'
sum_table = pd.read_csv(sum_table_dir)
PS_stats = pd.read_csv(PS_stats_dir)
        
a=0
for obsv in tqdm(id_set):
    
    #file = (base_directory + 'sum_tables/'+str(obsv)+'_summary.pdf')
    file = (som_directory +str(obsv)+'_'+str(som_size)+'x'+str(som_size)+'SOMs.csv')
    
    if os.path.exists(file):
        fig, ax = plt.subplots(1, 2, figsize=(9.5, 4))
        img1 = np.zeros(som_size*som_size).reshape((som_size, som_size))
        img2 = np.zeros(som_size*som_size).reshape((som_size, som_size))

        for nx in range(0,som_size):
            for ny in range(0,som_size):
                #print(nx, ny)
                directory = save_directory + 'excess_tables/'
                os.makedirs(directory, exist_ok=True)
                # iterate over files in
                # that directory
                excess1 = calculate_normalised_excess(obsv, sum_table, PS_stats, par1, nx, ny)
                excess2 = calculate_normalised_excess(obsv, sum_table, PS_stats, par2, nx, ny)

                tmp = pd.DataFrame({
                    'csd': obsv,
                    'winning_Nx': nx,
                    'winning_Ny': ny,
                    'excess_'+par1: excess1, 
                    'excess_'+par2: excess2,
                },
                index=[a])
                a = a+1
                #print(a, summ, avgg, stdd, excess1)

                img1[9-nx,ny] = excess1
                img2[9-nx,ny] = excess2

                excess = pd.concat([excess, tmp], ignore_index=True) 

        im1 = ax[0].imshow(img1, extent=[0,som_size,0,som_size], vmax = 3, vmin = -3, cmap='RdBu_r')
        im2 = ax[1].imshow(img2, extent=[0,som_size,0,som_size], vmax = 3, vmin = -3, cmap='RdBu_r')

        ax[0].set_xlabel('Neuron_x')
        ax[0].set_ylabel('Neuron_y')
        ax[0].set_title(par1)
        cbar1 = plt.colorbar(im1, ax=ax[0])
        cbar1.set_label('normalised excess')

        ax[1].set_xlabel('Neuron_x')
        ax[1].set_ylabel('Neuron_y')
        ax[1].set_title(par2)
        cbar2 = plt.colorbar(im2, ax=ax[1])
        cbar2.set_label('normalised excess')
        fig.suptitle('csd'+str(obsv))
        fig.subplots_adjust(left=0.08, bottom=0.13, right=0.95, top=0.9, wspace=0.26, hspace=0.1)

        fig.savefig(directory +'csd'+str(obsv)+'_normalised_excess.pdf')
        fig.clf()
    else:
        continue
excess.to_csv(directory+'excess.csv') 

  0%|          | 0/335 [00:00<?, ?it/s]

/tmp/ipykernel_89846/3150740380.py:29: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(1, 2, figsize=(9.5, 4))


<Figure size 684x288 with 0 Axes>

<Figure size 684x288 with 0 Axes>

<Figure size 684x288 with 0 Axes>

...

<Figure size 684x288 with 0 Axes>

<Figure size 684x288 with 0 Axes>

<Figure size 684x288 with 0 Axes>

<Figure size 684x288 with 0 Axes>

<Figure size 684x288 with 0 Axes>

In [5]:

def filter_SOM(obsv, somsize, nx, ny, training_no=training_no, revision=revision):

    filename = '/project/rpp-chime/hgan/ML-analysis/delaysp/'+revision+'/'+str(obsv)
    somfile = '../../ML-analysis_rev07/'+training_no+'/SOMs/'+revision+'/'+str(obsv)+'_'+str(somsize)+'x'+str(somsize)+'SOMs.csv'
    outdir = '/project/rpp-chime/hgan/ML-analysis_rev07/'+training_no+'/filtered/'+str(obsv)
    os.makedirs(outdir, exist_ok=True)
    outname = outdir + '/csd'+str(obsv)+'_filtered_'+str(nx)+','+str(ny)
    command1 = 'cd /project/rpp-chime/hgan/ML-analysis/software/disccofan-master/disccofan-SOM'

    command2 = './disccofan -g 1,1,1 --threads 1  -c 8  -a 12 \
    --inprefix %s --intype fits -o som --somsize %d --somfile %s --somneuron %d,%d \
    --outprefix %s --outtype fits'%(filename, somsize, somfile, nx, ny, outname)
    try: 
        #print(command2)
        os.system(command1+" && "+command2)

    except:
        print("something is wrong.....!!")

In [6]:
excess_table = pd.read_csv(save_directory+'/excess_tables/excess.csv',
                           sep=',', header=0, low_memory=False)

def find_excess_neurons(excess_table, par, nsig, obsv, somsize, training_no=training_no):
    par_mask = np.abs(excess_table['excess_'+par]) >= nsig
    nsigma_par = excess_table[par_mask].mask(np.isinf).dropna()
    obs_excess = nsigma_par[nsigma_par['csd'] == obsv]
    if obs_excess.empty :
        print("no excess reponse")
    else:
        obs_excess = obs_excess.reset_index()  # make sure indexes pair with number of rows
        for index, row in obs_excess.iterrows():
            nx = int(row['winning_Nx'])
            ny = int(row['winning_Ny'])
            print("filtering....csd"+str(obsv)+' ('+str(nx)+','+str(ny)+')')
            filter_SOM(obsv, somsize, nx, ny, training_no)

            
def plot_excess_neurons(excess_table, par, nsig, obsv, somsize, training_no=training_no):
    par_mask = np.abs(excess_table['excess_'+par]) >= nsig
    nsigma_par = excess_table[par_mask].mask(np.isinf).dropna()
    obs_excess = nsigma_par[nsigma_par['csd'] == obsv]
    base_directory = save_directory

    if obs_excess.empty :
        print("no excess reponse")
    else:
        obs_excess = obs_excess.reset_index()  # make sure indexes pair with number of rows
        num_figs = len(obs_excess) + 1
        
        n_column = 3
        n_yaxis = num_figs//n_column + 1
        
        width = 5
        height = 2.5
        fig, axes = plt.subplots(n_yaxis, n_column, figsize=(width*n_column, height*n_yaxis), 
                                 sharex=True, sharey=True)      
        kwargs = {"extent": (0, 1700, -1.28, 1.28), "vmin": -8, "vmax": 0, 
                  "aspect": "auto", "interpolation": "nearest", "cmap": 'inferno'}
        ori_ps = '/project/rpp-chime/hgan/ML-analysis/delaysp/'+revision+'/'+str(obsv)+'.fits'
        print(ori_ps)
        header = fits.open(ori_ps)
        image = header[0].data
        
        try:
            im = axes[0,0].imshow(image, **kwargs)
            axes[0,0].set_title('delay spectrum csd'+str(obsv))
        except:
            im = axes[0].imshow(image, **kwargs)
            axes[0].set_title('delay spectrum csd'+str(obsv))
        aa = 1 
        
        for index, row in obs_excess.iterrows():
            print(index)
            nx = int(row['winning_Nx'])
            ny = int(row['winning_Ny'])
            
            outdir = save_directory + 'filtered/'+str(obsv)
            filtered_ps = outdir + '/csd'+str(obsv)+'_filtered_'+str(nx)+','+str(ny)+'.fits'
    
            header = fits.open(filtered_ps)
            image = header[0].data
            #img_masked = np.ma.masked_where( np.abs(image)== 0, image)

            
            x_i = aa//n_column
            y_i = aa%n_column
            
            try:
                im = axes[x_i,y_i].imshow(image, **kwargs) #gxhat[0, :, :], aspect="auto", interpolation="nearest", vmin=0.5, vmax=1.5) 
                axes[x_i,y_i].set_title('neuron ('+str(nx)+','+str(ny)+') filtered')#"CSD"+str(obsv)+', '+str(csd_to_datetime(obsv)), 
                                        #x=.5, y=.99, fontsize=12, color="black")
                if y_i== n_column-1:
                    fig.colorbar(im, ax=axes[x_i,y_i])
            except:
                im = axes[y_i].imshow(image, **kwargs) #gxhat[0, :, :], aspect="auto", interpolation="nearest", vmin=0.5, vmax=1.5) 
                axes[y_i].set_title('neuron ('+str(nx)+','+str(ny)+') filtered')#"CSD"+str(obsv)+', '+str(csd_to_datetime(obsv)), 
                                        #x=.5, y=.99, fontsize=12, color="black")
                if y_i== n_column-1:
                    fig.colorbar(im, ax=axes[y_i])                
            #print("....."+str(round(i/no_figs*100))+"%......")
            aa = aa+1
        fig.subplots_adjust(right=0.92, left=0.12, wspace=0.1, hspace=0.15, top=0.87, bottom=0.15)
        #cbar_ax = fig.add_axes([0.90, 0.15, 0.02, 0.7])
        #fig.colorbar(im, cax=cbproj_dir)
        os.makedirs(save_directory+'excess_filtered/', exist_ok=True)
        fig.savefig(save_directory+'excess_filtered/'+str(obsv)+'_'+par+'.pdf')
        fig.clf()


In [7]:
def find_good_data(id_set, excess_table, par, nsig, somsize):
    good_nights=[]
    bad_nights=[]
    no_excess=[]
    for obsv in tqdm(id_set): 
        par_mask = np.abs(excess_table['excess_'+par]) >= nsig
        nsigma_par = excess_table[par_mask].mask(np.isinf).dropna()
        obs_excess = nsigma_par[nsigma_par['csd'] == obsv]
        if obs_excess.empty :
            good_nights.append(obsv)
        else:
            obs_excess = obs_excess.reset_index()  # make sure indexes pair with number of rows
            if len(obs_excess.index) < 5:
                good_nights.append(obsv)
            else:
                bad_nights.append(obsv) 
                no_excess.append(len(obs_excess.index))                
    return good_nights, bad_nights, no_excess

def find_a_neuron(id_set, excess_table, par, nsig, somsize, nx, ny):
    excess_par = []
    xxmask = excess_table['winning_Nx'] == nx
    yymask = excess_table['winning_Ny'] == ny
    par_mask = np.abs(excess_table['excess_'+par]) >= nsig
    nsigma_par = excess_table[par_mask&xxmask&yymask].mask(np.isinf).dropna()
    selected = nsigma_par['csd'].astype(int)
    excess_par = np.array(excess_table['excess_'+par])
    
    return np.array(selected), excess_par

In [8]:
def plot_nights(sub_set, excess, opt, revision=revision ):
    size = len(sub_set)
    n_x = 5
    n_y = 11
    nfigs = n_x * n_y
    total_iter = round (size / nfigs +0.5)
    base_directory = save_directory

    daily_processing_path = Path(f"/project/rpp-chime/chime/chime_processed/daily/{revision}/")

    def lscale(x):
        x = np.abs(x)
        return np.log10(x / x.max())

    kwargs = {"extent": (0, 1700, -1.28, 1.28), "vmin": -12, "vmax": 0}

    # for itera in range(0, total_iter) :
    fig, axes = plt.subplots(n_y, n_x, figsize=(5*n_x, 3*n_y), 
                             sharex=True, sharey=True, gridspec_kw={"wspace": 0.01, "hspace": 0.01})
    for ii, axis in enumerate(axes.ravel()):
        #print(ii)
        if ii < size :

            ex = str(sub_set[ii]) 
            excess_val = excess[ii]
            name = "delayspectrum_lsd_" + ex + ".h5"

            ds = containers.DelaySpectrum.from_file( daily_processing_path / ex / name)
            axis.imshow( lscale(ds.spectrum[:].T), **kwargs)
            axis.text(100, 1, f"{ex}", bbox=dict(facecolor="white", edgecolor=None))
            axis.text(420, 1, f"{excess_val}", color='white')

        # else:
        #     continue
    fig.savefig(base_directory+ 'filtered_'+opt+'_delayspectra.pdf')
    fig.clf()

In [9]:
def evaluate_nights(id_set, excess_table, par, somsize):
    good_nights=[]
    okay_nights=[]
    bad_nights=[]
    n_exc_good = []
    n_exc_okay = []
    n_exc_bad = []
    for obsv in tqdm(id_set): 
        par_mask1 = (excess_table['excess_'+par]) >= 1.0
        par_mask2 = (excess_table['excess_'+par]) >= 2.0
        par_mask3 = (excess_table['excess_'+par]) >= 3.0
        
        nsigma1_par = excess_table[par_mask1].mask(np.isinf).dropna()
        nsigma2_par = excess_table[par_mask2].mask(np.isinf).dropna()
        nsigma3_par = excess_table[par_mask2].mask(np.isinf).dropna()
        
        obs_excess1 = nsigma1_par[nsigma1_par['csd'] == obsv]
        obs_excess2 = nsigma2_par[nsigma2_par['csd'] == obsv]
        obs_excess3 = nsigma3_par[nsigma3_par['csd'] == obsv]
        
        if len(obs_excess1.index) < 10 and len(obs_excess2.index) < 10 and len(obs_excess3.index) < 5:
            good_nights.append(obsv)
            n_exc_good.append(len(obs_excess1.index))
        else:
            obs_excess1 = obs_excess1.reset_index()  # make sure indexes pair with number of rows
            if len(obs_excess1.index) < 15 and len(obs_excess2.index) < 15 and len(obs_excess3.index) < 5:
                okay_nights.append(obsv)
                n_exc_okay.append(len(obs_excess1.index))
            else:
                bad_nights.append(obsv) 
                n_exc_bad.append(len(obs_excess1.index))   
                
    return good_nights, okay_nights, bad_nights, n_exc_good, n_exc_okay, n_exc_bad

In [55]:
training_no = 'train1'
excess_table = pd.read_csv(save_directory+'/excess_tables/excess.csv')

In [59]:
all_set = id_set
good, okay, bad, num_exc1, num_exc2, num_exc3 = evaluate_nights(all_set, excess_table, 'flux', 10)

  0%|          | 0/335 [00:00<?, ?it/s]

In [60]:
print(np.array(good))
print(num_exc1)
print(np.array(okay))
print(num_exc2)
print(np.array(bad))
print(num_exc3)

[1868 1869 1870 1871 1872 1878 1879 1880 1889 1890 1893 1894 1907 1915
 1917 1924 1926 1928 1931 1932 1933 1934 2012 2013 2029 2031 2032 2033
 2039 2040 2041 2044 2046 2047 2060 2072 2073 2074 2075 2081 2084 2085
 2086 2087 2093 2094 2095 2096 2097 2098 2099 2105 2106 2107 2108 2109
 2111 2112 2115 2116 2118 2119 2120 2121 2122 2123 2126 2127 2132 2133
 2134 2142 2143 2163 2164 2166 2167 2174 2175 2179 2180 2181 2182 2186
 2193 2228 2473 2516 2522 2564 2788 2816 2823 2830 2837 2844 2851 2872
 2914 3012 3145 3201 3229 3341 3348 3362 3369 3376 3516 3517 3518 3520
 3521 3522 3523 3524 3527 3528 3530 3531]
[7, 5, 6, 5, 4, 8, 6, 7, 7, 9, 7, 8, 9, 6, 8, 9, 7, 9, 7, 8, 4, 6, 9, 8, 9, 5, 6, 7, 8, 9, 5, 7, 6, 8, 4, 5, 7, 3, 7, 4, 6, 5, 6, 3, 6, 4, 8, 3, 4, 3, 5, 4, 5, 5, 5, 6, 4, 5, 8, 4, 8, 4, 6, 7, 4, 5, 6, 6, 6, 6, 6, 8, 7, 6, 6, 6, 5, 7, 6, 6, 6, 7, 7, 6, 7, 8, 8, 7, 8, 9, 7, 9, 5, 9, 7, 7, 6, 5, 9, 9, 9, 7, 9, 9, 7, 6, 5, 7, 7, 7, 9, 8, 8, 7, 7, 6, 7, 6, 4, 6]
[1886 1892 1903 1904 1906 190

In [61]:
plot_nights(good,  num_exc1,'good', )#training_no)
plot_nights(okay, num_exc2, 'okay', )#training_no)
plot_nights(bad, num_exc3, 'bad', )#training_no)

/tmp/ipykernel_89846/2130590213.py:13: RuntimeWarning: divide by zero encountered in log10
  return np.log10(x / x.max())
/tmp/ipykernel_89846/2130590213.py:13: RuntimeWarning: divide by zero encountered in log10
  return np.log10(x / x.max())
/tmp/ipykernel_89846/2130590213.py:13: RuntimeWarning: divide by zero encountered in log10
  return np.log10(x / x.max())


<Figure size 1800x2376 with 0 Axes>

<Figure size 1800x2376 with 0 Axes>

<Figure size 1800x2376 with 0 Axes>

In [10]:
from chimedb import core
from chimedb import dataflag as df
from ch_util import ephemeris
from ch_util import tools

from caput import time as ctime
from draco.core import containers
from draco.analysis.beamform import icrs_to_cirs
from ch_util import rfi
from ch_pipeline.core import containers as ccontainers

matplotlib.rc('figure', figsize=(9,6))
matplotlib.rc('axes', edgecolor='k')
matplotlib.rc('grid', color='k', linewidth=0.5, linestyle=':')
matplotlib.rc('font', size=12)
matplotlib.rc('image', aspect='auto', interpolation='nearest', cmap='inferno')

In [12]:

#csd = good_data[0]
som_size=10
cubes = []
for csd in id_set:
    par = 'flux'
    img1 = np.zeros(som_size*som_size).reshape((som_size, som_size))
    for nx in range(0,som_size):
        for ny in range(0,som_size):
            mask_csd = excess_table['csd'] == csd
            mask_x = excess_table['winning_Nx'] == nx
            mask_y = excess_table['winning_Ny'] == ny
            img1[9-nx,ny] = excess_table[mask_csd&mask_x&mask_y]['excess_'+par].values[0]
    cubes.append(img1)



In [13]:
imgs = []
for csd in id_set:
    par = 'flux'
    img1 = np.zeros(som_size*som_size).reshape((som_size, som_size))
    for nx in range(0,som_size):
        for ny in range(0,som_size):
            mask_csd = excess_table['csd'] == csd
            mask_x = excess_table['winning_Nx'] == nx
            mask_y = excess_table['winning_Ny'] == ny
            img1[9-nx,ny] = excess_table[mask_csd&mask_x&mask_y]['excess_'+par].values[0]
    imgs.append(img1)

In [ ]:
size = len(id_set)
n_x = 10
n_y = 34
nfigs = n_x * n_y
total_iter = round (size / nfigs +0.5)

kwargs = {"extent": (0,som_size,0,som_size), "vmin": -3, "vmax": 3, "cmap":'RdBu_r'}


# for itera in range(0, total_iter) :
fig, axes = plt.subplots(n_y, n_x, figsize=(1*n_x, 1.3*n_y), 
                         sharex=True, sharey=True, gridspec_kw={"wspace": 0.01, "hspace": 0.3})
for ii, axis in enumerate(axes.ravel()):
    #print(ii)
    if ii < size :
        ex = str(id_set[ii]) 
        name = "csd" + ex
        axis.imshow(cubes[ii], **kwargs)
        axis.set_title(name, fontsize=10)
        #axis.text(100, 1, f"{ex}", bbox=dict(facecolor="white", edgecolor=None))
        #axis.text(420, 1, f"{excess_val}", color='white')